# Lab 3: Classification 

In this lab, you will gain experience with several classification algorithms, including Logistic Regression, Naive Bayes, and Support-Vector Machines. You will also work towards building a modular machine learning pipeline that you can use for your class project and hopefully many other projects in the future. Finally, you will gain experience with geospatial data analysis.

You will work with the crime data from Chicago's Open Data Portal to better understand the crimes that are reported in different neighborhoods in the city, whether those crimes result in arrest, and so forth. As in previous labs, you will acquite the data and familiarize yourself with its basic properties. You will then explore a few classification problems using different machine learning models.

## Learning Objectives

* Starting to build a reusable data science / machine learning analysis pipeline 
* Creating functions with fixed modules and steps that can be reused for your final project
* Writing several `scikit-learn` features by hand, like `GridSearchCV`
* Applying the following classification models (essentially plugging different functions into the same pipeline):
    * Logistic Regression
    * Naive Bayes
    * Support-Vector Machines

In [1]:
# YOUR CODE HERE

## Part 1: Data Acquisition and Exploratory Analysis

In the first part of the lab, you will acquire data from the Chicago Open Data Portal concerning crime statistics and perform basic exploration and analysis.

### 1.1 Data Download

Download [reported crime data for 2019](https://data.cityofchicago.org/Public-Safety/Crimes-2019/w98m-zvie) from the Chicago Open Data Portal. Your dataset should have roughly 260,000 observations. As always, think about what each row represents and familiarize yourself with the variables. 

In [2]:
# YOUR CODE HERE

### 1.2 Basic Exploratory Analysis

Before delving into our analysis, we want to explore the reported crime data. Your task here is to generate visualizations for each question, exploring the number of crimes of each type, how these change over time, and how they vary across neighborhoods. Note that this is intentionally open ended – produce visualizations that you think would be meaningful. 

Each of your visualizations should include a title, labeled axes, and a brief (2-3 sentence) description.

#### 1.2.1 How many crimes of each type are in these data? 

In [3]:
# YOUR CODE HERE

#### 1.2.2 Is there any notable seasonality in the number of reported crimes? 

You can perform this analysis at the month or season level.

In [4]:
# YOUR CODE HERE

#### 1.2.3 How did reported crimes vary across neighborhoods? 

You may find the [neighborhood boundaries data](https://data.cityofchicago.org/Facilities-Geographic-Boundaries/Boundaries-Neighborhoods/bbvz-uum9) from the Chicago Data Portal and the `geopandas` [documentation on spatial joins](https://geopandas.org/mergingdata.html) useful here. A few more helpful resources can be found [here](https://gis.stackexchange.com/questions/174159/convert-a-pandas-dataframe-to-a-geodataframe) and [here](https://towardsdatascience.com/lets-make-a-map-using-geopandas-pandas-and-matplotlib-to-make-a-chloropleth-map-dddc31c1983d).

Your data preparation process will likely include the following steps:
1. Load the neighborhood boundaries `geojson` file using `geopandas`.
2. Convert your crime dataframe to a `geopandas` dataframe.
3. Merge the crime `geopandas` dataframe and the neighborhoods `geopandas` dataframes.

In [5]:
# YOUR CODE HERE

### 1.3 Joining with Census Data

The crime dataset includes lat/long fields, which can be joined with Census data. Your task now is to augment that data with [American Community Survey data](https://www.census.gov/programs-surveys/acs/data.html). For each crime report, use one of the census APIs to get additional data about the demographics of the block where the crime report came from. At a minimum, this should include the percentage of each block that is African American/Black (`Percent Black`) and White (`Percent White`), along with the median income in each block (`Median Income`). 

We recommend using [ACS 5-Year Data](https://www.census.gov/data/developers/data-sets/acs-5year.html), which is estimated for all geographies down to the block-group level. You may find the following resources helpful in constructing your API call: [ACS API Variable Formats](https://www.census.gov/data/developers/data-sets/acs-1year/notes-on-acs-api-variable-formats.html) and [Census Query Examples](https://www.census.gov/data/developers/guidance/api-user-guide/query-examples.html). The [Census Block TIGER boundaries](https://data.cityofchicago.org/dataset/CensusBlockTIGER2010/bt9m-d2mf/data) from the Chicago Data Portal may also be helpful in performing a spatial join between the ACS data and the crime data. 

We've included an example using the `CensusData` API for a single field: B02001_001E (Total Population). In this example, we download the non-spatial block-level ACS data and merge it with the spatial block-level boundaries. We also create a quick map to verify that our output resembles Chicago. Feel free to leverage this code to pull your additional variables. 

In [6]:
# --- Example of downloading ACS data from the CensusData API --- # 

import censusdata
import geopandas as gpd

# Note: IL FIPS = 17, Cook County FIPS = 031, Table = B02001_001E (Total Population)
acs_example = censusdata.download("acs5", 2015, censusdata.censusgeo(
    [("state", "17"), ("county", "031"), ("block group", "*")]), ["B02001_001E", "GEO_ID"])

# Download Census block boundaries for Chicago 
census_gdf = gpd.read_file("https://data.cityofchicago.org/resource/bt9m-d2mf.geojson?$limit=9999999")

# Extract 12-digit FIPS code from both datasets 
census_gdf["geo_12"] = census_gdf["geoid10"].map(lambda x: str(x)[:12])
acs_example["geo_12"] = acs_example["GEO_ID"].map(lambda x: str(x)[-12:])

# Merge ACS data with Census block boundaries 
# Assumes the crime df was converted to a geopandas df (crime_gdf)  
merged_gdf = (gpd.GeoDataFrame(acs_example.merge(census_gdf, on="geo_12", how="inner"), 
                               crs=crime_gdf.crs))

# Limit columns 
limited_gdf = merged_gdf[["B02001_001E", "GEO_ID", "geometry"]].drop_duplicates()

# Plot geopandas dataframe with total population by Census block
limited_gdf.plot()

# Print a sample of rows 
limited_gdf.sample(3)

#### 1.3.1 Load ACS data, TIGER boundaries, and merge to crime data
Using the code provided above as a starting point, take the following steps in your data preparation:

1. Load the ACS data with at least variables for `Total Population`, `White`, `Black`, and `Median Income`.
2. Load the TIGER neighborhood boundaries `geopandas` dataframe.
3. Merge the ACS dataframe and the TIGER `geopandas` dataframe. The result will be a `geopandas` dataframe.
4. Merge the ACS `geopandas` dataframe that you created in step 3 with your crime `geopandas` dataframe from section 1.2.3 above.
5. Compute the percentage of each Census block's population that is White and African American/Black.

Based on this augmented data, explore the characteristics of blocks with crime reports. For instance, you might look into demographic trends as they relate to homicides or total reported crimes. Provide descriptive statistics and produce at least one visualization. Again, include a title, labeled axes, and a brief description.

In [7]:
# YOUR CODE HERE

### 1.4 Basic Bayesian Probability

Assume you are running the 911 call center for Chicago. You get a call from someone at 5801 S Ellis Ave.  In this section, we ask several basic questions. Produce your answers by writing code against the dataframes above.

You are also welcome to express your answer mathematically. You can use LaTeX syntax in notebooks to write equations.

**1.4.1. Which crime type is the most likely given the call came from 5801 S Ellis Ave? (What are the probabilities for each type of crime?)**

In [8]:
# YOUR CODE HERE

**1.4.2. Let’s now assume that a call comes in about Theft. Which is more likely – that the call came from Logan Square or Hyde Park? How much more or less likely is it to be from Logan Square vs. Hyde Park?**

You can consult [this link](https://en.wikipedia.org/wiki/Community_areas_in_Chicago) to see a list of community areas and their numbers, which corresponds to the `Community Area` field in the crime dataframe.

In [9]:
# YOUR CODE HERE

**1.4.3. Now assume that you don’t have access to all the raw data and you know the following facts:**

* There are a total of 2000 calls, 800 from Garfield Park and 500 from Uptown. 
* Of the 800 calls from Garfield Park, 200 of them are about Battery. 
* Of the 500 calls from Uptown, 160 are about Battery. 

If a call comes about Battery, how much more/less likely is it that the call came from Garfield Park versus Uptown?

In [10]:
# YOUR CODE HERE

---

## Part 2: Machine Learning Pipeline

When working on supervised machine learning projects, you will find yourself repeatedly performing tasks with slight variations. This naturally lends itself to a pipeline – or a set of functions that can help automate your workflow. The goal of this section is to start to build a simple, modular, and extensible pipeline, and then to apply this pipeline to the classification problem described above (i.e. predicting whether an arrest occured).

**Do NOT use `sklearn.pipeline.Pipeline()` here**. An important objective of this assignment is to understand how this process works and the ways different steps can be customized when designed from scratch.

You may find it useful to read through Part 2 in its entirety before beginning. You will use the code you write for your pipeline in Part 2.1 to answer the questions in Part 2.2.

### 2.1 Building the Pipeline 

To reiterate, the objective here is to build a simple and modular pipeline that can be extended throughout the quarter. Your framework should include the components described below, but is likely to evolve as you begin applying it to different machine learning tasks. Your functions should be well-documented and appropriately named. Focus on the structure, modularity, and extensibility. 

Your functions should be written in a separate `pipeline.py` file that you import into this (and future) notebooks. At a minimum, include the following components – 

1. **Read Data**  
Write 1 function in `pipeline.py` to load the data. Your function for reading in data can be as simple as calling `pd.read_csv`. If this step is more complicated (e.g. in your projects), you will want to write more detailed functions.
2. **Explore Data**  
Write at least 2 functions in `pipeline.py` to automate common exploratory tasks. This can include generating distributions of variables, correlations between them, identifying outliers, summarizing by groups, identifying the time range of the data, etc. Feel free to leverage your work from previous labs and Step 1 above.
3. **Create Training and Testing Sets**  
Write 1 function in `pipeline.py` to create training and testing splits. For this assignment, this step can be as simple as calling `sklearn.model_selection.train_test_split`. Later on, you should use a separate training set, validation set (to tune hyperparameters), and testing set to perform cross-validation.  
4. **Pre-Process Data**  
Write 2 functions in `pipeline.py` to automate pre-processing steps. One function should impute missing values of continuous variables using the median value and the other should normalize continuous variables. 
5. **Generate Features**   
Write 2 functions in `pipeline.py` to faciliate feature generation. One function should perform one-hot encoding of categorical variables (e.g. with `pd.get_dummies`) and one function should discretize continuous variables (e.g. with `pd.cut`). Discretizing continuous variables can be useful in cases where the variable has a significant cutoff value (for example, age could be discretized to distinguish between children under 18 and adults 18 and older).
6. **Build Classifiers**   
Write 1 function in `pipeline.py` that applies at least one machine learning model to a dataset. The function should also print the amount of time required to train each model. Several `scikit-learn` methods will be useful here, including `set_params` and `model_selection.ParameterGrid`. 
7. **Evaluate Classifiers**  
Write 1 function in `pipeline.py` that calculates the accuracy of your models based on your testing set. Again, you do NOT need a third validation set at this point in the quarter.

Import your pipeline into this notebook in the cell below. 

In [11]:
import pipeline as pipeline 

### 2.2 Applying the Pipeline 

Your task is to build a classifier that predicts the binary outcome of whether an arrest occured in each report (`Arrest`). Your classifier should take the following features as inputs: 
- `Primary Type` 
- `Ward` 
- `FBI Code`
- `Percent White` 
- `Percent Black`
- `Median Income`

Note that the three features from the crime data (`Primary Type`, `Ward`, and `FBI Code`) are categorical, while the three features from the ACS data (`Percent White`, `Percent Black`, and `Median Income`) are continuous. 

Train and evaluate the following types of classifiers: 
- Logistic Regression (`sklearn.linear_model.LogisticRegression`) 
- Support Vector Machines (`sklearn.svm.LinearSVC`)
- Naive Bayes (`sklearn.naive_bayes.GaussianNB`)

Use the helper functions that you wrote in your `pipeline.py` file to work through this classification task. The majority of the work in each cell should be performed by your `pipeline.py` functions, but you can add some extra code in this notebook if need be.

#### 2.2.1 Read Data (Call from Part 1)
We will be using the dataset compiled in Step 1. Feel free to drop any extra columns that you may have acquired in merging datasets above, and print out the first few rows of this dataset. 

In [12]:
# YOUR CODE HERE

#### 2.2.2 Explore Data
Apply one of the exploration functions in your pipeline on this dataset. 

In [13]:
# YOUR CODE HERE

#### 2.2.3 Create Training and Testing Sets
Recall that all pre-processing steps must be done separately on your training and testing data. As a result, you should split your data into training and testing sets at the onset. Print out the number of observations in your training and testing sets. 

In [14]:
# YOUR CODE HERE

#### 2.2.4 Pre-Process Data & Generate Features

We will do these steps separately for the label, the numeric features, and the categorical features.  

First, convert the label `Arrest` into a numerical (rather than boolean) feature in both your training and testing data. Print the percentage of recorded crimes in your training data that resulted in an arrest. 

In [15]:
# YOUR CODE HERE

Next, pre-process the continuous numeric features (`Percent White`, `Percent Black`, `Median Income`). This should include the following steps: 
1. Impute missing values. Here, feel free to simply use the mean or the median of the column. 
2. Normalize each feature to have a mean of zero and standard deviation of one. Recall that this should be done based on the training data. 

Note that administrative data often uses encodings to indicate missing data. Always be sure to perform sanity checks (e.g. ensure that your percentages fall between 0 and 1, that income follows a reasonable distribution, etc.) 

In [16]:
# YOUR CODE HERE

Finally, encode features from the categorical variables (`Primary Type`, `Ward`, and `FBI Code`). The standard way to encode categorical features in machine learning is through one-hot encoding. The function `pd.get_dummies` will be useful. 

An inherent issue arises with this approach when a value appears in either your training or testing data, but not in both. If a value appears in your training set but not your testing set, create a column with all 0's in your testing set. If a value appears in your testing set but not your training set, drop it from your testing data. 

In [17]:
# YOUR CODE HERE

We now have a modeling dataset that we can use to train and evaluate classifiers. Print out the first few rows of your training dataset. At this point, your dataset should only include the features created above and the label `Arrest`. 

In [18]:
# YOUR CODE HERE

We've provided a `sanity_check` function below to help check these steps. Pass your training and testing sets through this function to confirm that your features match across your training and testing data and that you have no missing values. 

In [19]:
def sanity_check(train_df, test_df): 
    
    # Sort features alphabetically
    train_df = train_df.reindex(sorted(train_df.columns), axis=1)
    test_df = test_df.reindex(sorted(test_df.columns), axis=1)

    # Check that they have the same features
    if (train_df.columns == test_df.columns).all():
        print("Success: Features match")

    # Check that no NAs remain
    if  not train_df.isna().sum().astype(bool).any() and \
        not test_df.isna().sum().astype(bool).any():
        print("Success: No NAs remain")
        
# YOUR CODE HERE

#### 2.2.5 Build and Evaluate Classifiers  

Train and evaluate models using the following types of classifiers and hyperparameters set to several values:
- Logistic Regression (`sklearn.linear_model.LogisticRegression`): `penalty`, `C`  
- Support Vector Machines (`sklearn.svm.LinearSVC`): `C`
- Naive Bayes (`sklearn.naive_bayes.GaussianNB`)

We've provided the dictionaries `MODELS` and `GRID` below that specify the 16 models that you should run and the hyperparameter values to consider (10 Logistic Regressions, 1 Naive Bayes, and 5 SVM models). 

We've also provided pseudocode to perform a manual grid search across these parameters. This function should be in your `pipeline.py` file and then called in this notebook. The `set_params(**params)` method [here](https://scikit-learn.org/stable/modules/generated/sklearn.base.BaseEstimator.html#sklearn.base.BaseEstimator.set_params) will be particularly useful.

In [20]:
# --- Setup  --- # 
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
import datetime

# Config: Dictionaries of models and hyperparameters
MODELS = {
    'LogisticRegression': LogisticRegression(), 
    'LinearSVC': LinearSVC(), 
    'GaussianNB': GaussianNB()
}

GRID = {
    'LogisticRegression': [{'penalty': x, 'C': y, 'random_state': 0} 
                           for x in ('l2', 'none') \
                           for y in (0.01, 0.1, 1, 10, 100)],
    'GaussianNB': [{'priors': None}],
    'LinearSVC': [{'C': x, 'random_state': 0} \
                  for x in (0.01, 0.1, 1, 10, 100)]
}


# --- Grid Search Pseudocode --- # 
# Move this into a function in your pipeline.py file! 

# Begin timer 
start = datetime.datetime.now()

# Initialize results data frame 
# YOUR CODE HERE

# Loop over models 
for model_key in MODELS.keys(): 
    
    # Loop over parameters 
    for params in GRID[model_key]: 
        print("Training model:", model_key, "|", params)
        
        # Create model 
        model = MODELS[model_key]
        model.set_params(**params)
        
        # Fit model on training set 
        # YOUR CODE HERE
        
        # Predict on testing set 
        # YOUR CODE HERE
        
        # Evaluate predictions 
        # YOUR CODE HERE
        
        # Store results in your results data frame 
        # YOUR CODE HERE
        
# End timer
stop = datetime.datetime.now()
print("Time Elapsed:", stop - start)        

### 2.3 Summarizing the Classification Models

#### 2.3.1. Summarize performance across your models. 
Produce a table summarizing each of your model specifications. Your columns should include the type of classifier (i.e. Logistic Regression, SVM, or Naive Bayes), the parameters for that iteration (e.g. `{'C': 0.1, 'penalty': 'l1'}`), and the evalaution metrics that you selected. Recall that this table should include 16 rows – or one for each model specification. 

In [21]:
# YOUR CODE HERE

#### 2.3.2 Describe the best performing model. 

Based on the table above, what type of classifier performed best and with which set of hyperparameters? 

YOUR RESPONSE HERE

#### 2.3.3 Which features were most important in the best performing model? 

In [22]:
# YOUR CODE HERE

#### 2.3.4 Briefly describe your findings for a non-technical audience.  
Describe the machine learning question you addressed, the methods your used, and your results. Discuss the broader policy implications of your analysis and any relevant caveats. Limit your response to 5-7 sentences. 

YOUR RESPONSE HERE